## Recurrent Neural Network with LSTM layers
### Remixed into PyTorch from

https://machinelearningmastery.com/develop-word-based-neural-language-models-python-keras/

In [ ]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
def onehot(values, num_classes):
    output = torch.zeros((len(values), num_classes))
    output[np.arange(len(values)), values] = 1.
    return output

In [ ]:
def split_words(paragraph):
    return paragraph.replace('\n', ' ').lower().split()

In [ ]:
def generate_encoding(tokens):
    counts = {}
    for token in tokens:
        if token not in counts:
            counts[token] = 0
            
        counts[token] += 1
    
    sorted_tokens = sorted(counts.keys(), key=counts.get, reverse=True)

    encoding = dict(zip(sorted_tokens, range(1, 1+len(sorted_tokens))))

    return encoding

def tokenize(tokens, encoding):
    output = []
    for token in tokens:
        output.append(encoding[token])
    return output

In [ ]:
def pre_pad(sequences, max_length, pad_value=0):
    padded = torch.full((len(sequences), max_length), pad_value, dtype=torch.long)
    for i, sequence in enumerate(sequences):
        padded[i, -len(sequence):] = torch.Tensor(sequence)

    return padded

### Model 1: One word in, one word out

In [ ]:
def word_by_word_seq(model, encoding, seed_text, n_words):
    curr_text, result = seed_text, seed_text

    for _ in range(n_words):
        # Text -> Int
        encoded = tokenize(split_words(curr_text), encoding)
        encoded = torch.Tensor(encoded).reshape((1, -1)).to(torch.long)

        # predict word
        word_vec = model.forward(encoded).argmax()

        # predicted idx -> word
        for word, idx in encoding.items():
            if idx == word_vec:
                curr_text = word
                break

        #
        result += f" {curr_text}"

    return result

In [ ]:
data = """ Jack and Jill went up the hill\n
		To fetch a pail of water\n
		Jack fell down and broke his crown\n
		And Jill came tumbling after\n """

# Text -> Integer
encoding = generate_encoding(split_words(data))
encoded = tokenize(split_words(data), encoding)

vocab_size = len(encoding) + 1
print(f"Vocabulary Size: {vocab_size}")

# Word : Word sequences
sequences = [encoded[i-1:i+1] for i in range(1, len(encoded))]
sequences = torch.Tensor(sequences).to(torch.long)
print(f"Total Sequences: {sequences.shape[0]}")

X, y = sequences[:, 0], sequences[:, 1]
y = onehot(y, num_classes=vocab_size)

In [ ]:
# Model
class RNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.l1 = nn.Embedding(vocab_size, 10)
        self.l2 = nn.LSTM(10, 50)
        self.l3 = nn.Linear(50, vocab_size)

    def forward(self, x):
        x = self.l1(x)
        x, _ = self.l2(x)
        x = F.tanh(x)
        x = F.sigmoid(self.l3(x))
        return x

model = RNN()
print(repr(model))

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=.001)

In [ ]:
# Train
for epoch in range(500):

    total_loss = 0
    for i, inputs in enumerate(X.reshape((-1, 1, 1))):
        labels = y[i].reshape((-1)).argmax().reshape(1)

        optimizer.zero_grad()

        outputs = model(inputs)
        outputs = outputs.reshape((1, -1))

        loss = criterion(outputs, labels.to(torch.long))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    if epoch % 25 == 24:
        print(f'{epoch+1:4.0f} | loss: {total_loss / 2000:.3f}')
        total_loss = 0.0

In [ ]:
#
print(word_by_word_seq(model, encoding, 'Jack', 6))

### Model 2: Line by line sequence

In [ ]:
def seq_to_word_seq(model, encoding, max_length, seed_text, n_words):
    in_text = seed_text

    for _ in range(n_words):
        encoded = tokenize(split_words(in_text), encoding)
        encoded = pre_pad([encoded], max_length)

        yhat = model.forward(encoded)
        print(yhat)
        out_word = ''
        for word, index in encoding.items():
            if index == yhat:
                out_word = word
                break

        in_text += f" {out_word}"

    return in_text

In [ ]:
print(seq_to_word_seq(model, encoding, max_length-1, 'Jack', 4))
print(seq_to_word_seq(model, encoding, max_length-1, 'Jill', 4))

In [ ]:
data = """ Jack and Jill went up the hill\n
		To fetch a pail of water\n
		Jack fell down and broke his crown\n
		And Jill came tumbling after\n """

# Text -> Integer
encoding = generate_encoding(split_words(data))
encoded = tokenize(split_words(data), encoding)

vocab_size = len(encoding) + 1
print(f"Vocabulary Size: {vocab_size}")

# Line based sequences
sequences = list()
for line in data.split('\n'):
    encoded = tokenize(split_words(line), encoding)
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))

# Pad
max_length = max([len(seq) for seq in sequences])
sequences = pre_pad(sequences, max_length)
print('Max Sequence Length: %d' % max_length)

X, y = sequences[:,:-1], sequences[:,-1]
y = onehot(y, num_classes=vocab_size)

In [ ]:
# Model
class RNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.l1 = nn.Embedding(vocab_size, 10)
        self.l2 = nn.LSTM(10, 50)
        self.l3 = nn.Linear(50, vocab_size)

    def forward(self, x):
        x = self.l1(x)
        x, _ = self.l2(x)
        x = F.tanh(x)
        x = F.sigmoid(self.l3(x))
        return x

model = RNN()
print(repr(model))

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=.001)

In [ ]:
# Train
for epoch in range(500):

    total_loss = 0
    for i, inputs in enumerate(X.reshape((-1, 1, max_length))):
        labels = y[i].reshape((-1)).argmax().reshape(1)

        optimizer.zero_grad()

        outputs = model(inputs)
        outputs = outputs.reshape((1, -1))

        loss = criterion(outputs, labels.to(torch.long))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    if epoch % 25 == 24:
        print(f'{epoch+1:4.0f} | loss: {total_loss / 2000:.3f}')
        total_loss = 0.0

In [ ]:
print(seq_to_word_seq(model, encoding, max_length-1, 'Jack', 4))
print(seq_to_word_seq(model, encoding, max_length-1, 'Jill', 4))